In [1]:
from tensorflow.keras import backend as K
from tensorflow.keras import activations, initializers
from tensorflow.keras.layers import Layer

import tensorflow as tf
import tensorflow_probability as tfp

tfd = tfp.distributions
tfp.__version__

'0.8.0'

In [2]:
def bnn_extractor(flat_observations, net_arch, act_fun):
    """
    Constructs an variational layer that receives observations as an input and outputs a latent representation for the policy and
    a value network. The ``net_arch`` parameter allows to specify the amount and size of the hidden layers and how many
    of them are shared between the policy network and the value network. It is assumed to be a list with the following
    structure:
    1. An arbitrary length (zero allowed) number of integers each specifying the number of units in a shared layer.
       If the number of ints is zero, there will be no shared layers.
    2. An optional dict, to specify the following non-shared layers for the value network and the policy network.
       It is formatted like ``dict(vf=[<value layer sizes>], pi=[<policy layer sizes>])``.
       If it is missing any of the keys (pi or vf), no non-shared layers (empty list) is assumed.
    For example to construct a network with one shared layer of size 55 followed by two non-shared layers for the value
    network of size 255 and a single non-shared layer of size 128 for the policy network, the following layers_spec
    would be used: ``[55, dict(vf=[255, 255], pi=[128])]``. A simple shared network topology with two layers of size 128
    would be specified as [128, 128].
    :param flat_observations: (tf.Tensor) The observations to base policy and value function on.
    :param net_arch: ([int or dict]) The specification of the policy and value networks.
        See above for details on its formatting.
    :param act_fun: (tf function) The activation function to use for the networks.
    :return: (tf.Tensor, tf.Tensor) latent_policy, latent_value of the specified network.
        If all layers are shared, then ``latent_policy == latent_value``
    """
    latent = flat_observations
    policy_only_layers = []  # Layer sizes of the network that only belongs to the policy network
    value_only_layers = []  # Layer sizes of the network that only belongs to the value network
    kernel_divergence_fn=lambda q, p, _: tfp.distributions.kl_divergence(q, p)

    # Iterate through the shared layers and build the shared parts of the network
    for idx, layer in enumerate(net_arch):
        if isinstance(layer, int):  # Check that this is a shared layer
            layer_size = layer
#             latent = act_fun(linear(latent, "shared_fc{}".format(idx), layer_size, init_scale=np.sqrt(2)))
            latent = act_fun(tfp.layers.DenseFlipout(layer_size, activation = 'relu', kernel_divergence_fn=kernel_divergence_fn)(latent))
        else:
            assert isinstance(layer, dict), "Error: the net_arch list can only contain ints and dicts"
            if 'pi' in layer:
                assert isinstance(layer['pi'], list), "Error: net_arch[-1]['pi'] must contain a list of integers."
                policy_only_layers = layer['pi']

            if 'vf' in layer:
                assert isinstance(layer['vf'], list), "Error: net_arch[-1]['vf'] must contain a list of integers."
                value_only_layers = layer['vf']
            break  # From here on the network splits up in policy and value network

    # Build the non-shared part of the network
    latent_policy = latent
    latent_value = latent
    for idx, (pi_layer_size, vf_layer_size) in enumerate(zip_longest(policy_only_layers, value_only_layers)):
        if pi_layer_size is not None:
            assert isinstance(pi_layer_size, int), "Error: net_arch[-1]['pi'] must only contain integers."
#             latent_policy = act_fun(linear(latent_policy, "pi_fc{}".format(idx), pi_layer_size, init_scale=np.sqrt(2)))
            latent_policy = act_fun(tfp.layers.DenseFlipout(pi_layer_size, activation = 'relu', kernel_divergence_fn=kernel_divergence_fn)(latent))

        if vf_layer_size is not None:
            assert isinstance(vf_layer_size, int), "Error: net_arch[-1]['vf'] must only contain integers."
#             latent_value = act_fun(linear(latent_value, "vf_fc{}".format(idx), vf_layer_size, init_scale=np.sqrt(2)))
            latent_value = act_fun(tfp.layers.DenseFlipout(vf_layer_size, activation = 'relu', kernel_divergence_fn=kernel_divergence_fn)(latent))

    return latent_policy, latent_value

In [3]:
from stable_baselines.common.policies import ActorCriticPolicy, nature_cnn

class FeedForwardPolicy(ActorCriticPolicy):
    """
    Policy object that implements actor critic, using a feed forward neural network.
    :param sess: (TensorFlow session) The current TensorFlow session
    :param ob_space: (Gym Space) The observation space of the environment
    :param ac_space: (Gym Space) The action space of the environment
    :param n_env: (int) The number of environments to run
    :param n_steps: (int) The number of steps to run for each environment
    :param n_batch: (int) The number of batch to run (n_envs * n_steps)
    :param reuse: (bool) If the policy is reusable or not
    :param layers: ([int]) (deprecated, use net_arch instead) The size of the Neural network for the policy
        (if None, default to [64, 64])
    :param net_arch: (list) Specification of the actor-critic policy network architecture (see mlp_extractor
        documentation for details).
    :param act_fun: (tf.func) the activation function to use in the neural network.
    :param cnn_extractor: (function (TensorFlow Tensor, ``**kwargs``): (TensorFlow Tensor)) the CNN feature extraction
    :param feature_extraction: (str) The feature extraction type ("cnn" or "mlp")
    :param kwargs: (dict) Extra keyword arguments for the nature CNN feature extraction
    """

    def __init__(self, sess, ob_space, ac_space, n_env, n_steps, n_batch, reuse=False, layers=None, net_arch=None,
                 act_fun=tf.tanh, cnn_extractor=nature_cnn, feature_extraction="cnn", **kwargs):
        super(FeedForwardPolicy, self).__init__(sess, ob_space, ac_space, n_env, n_steps, n_batch, reuse=reuse,
                                                scale=(feature_extraction == "cnn"))

        self._kwargs_check(feature_extraction, kwargs)

        if layers is not None:
            warnings.warn("Usage of the `layers` parameter is deprecated! Use net_arch instead "
                          "(it has a different semantics though).", DeprecationWarning)
            if net_arch is not None:
                warnings.warn("The new `net_arch` parameter overrides the deprecated `layers` parameter!",
                              DeprecationWarning)

        if net_arch is None:
            if layers is None:
                layers = [64, 64]
            net_arch = [dict(vf=layers, pi=layers)]

        with tf.variable_scope("model", reuse=reuse):
            if feature_extraction == "cnn":
                pi_latent = vf_latent = cnn_extractor(self.processed_obs, **kwargs)
            elif feature_extraction == "bnn":
                pi_latent, vf_latent = bnn_extractor(tf.layers.flatten(self.processed_obs), net_arch, act_fun)
            else:
                pi_latent, vf_latent = mlp_extractor(tf.layers.flatten(self.processed_obs), net_arch, act_fun)

            self._value_fn = linear(vf_latent, 'vf', 1)

            self._proba_distribution, self._policy, self.q_value = \
                self.pdtype.proba_distribution_from_latent(pi_latent, vf_latent, init_scale=0.01)

        self._setup_init()

    def step(self, obs, state=None, mask=None, deterministic=False):
        if deterministic:
            action, value, neglogp = self.sess.run([self.deterministic_action, self.value_flat, self.neglogp],
                                                   {self.obs_ph: obs})
        else:
            action, value, neglogp = self.sess.run([self.action, self.value_flat, self.neglogp],
                                                   {self.obs_ph: obs})
        return action, value, self.initial_state, neglogp

    def proba_step(self, obs, state=None, mask=None):
        return self.sess.run(self.policy_proba, {self.obs_ph: obs})

    def value(self, obs, state=None, mask=None):
        return self.sess.run(self.value_flat, {self.obs_ph: obs})

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [4]:
import warnings
from itertools import zip_longest
from abc import ABC, abstractmethod

import numpy as np
import tensorflow as tf
from gym.spaces import Discrete

from stable_baselines.common.tf_util import batch_to_seq, seq_to_batch
from stable_baselines.common.tf_layers import conv, linear, conv_to_fc, lstm
from stable_baselines.common.distributions import make_proba_dist_type, CategoricalProbabilityDistribution, \
    MultiCategoricalProbabilityDistribution, DiagGaussianProbabilityDistribution, BernoulliProbabilityDistribution
from stable_baselines.common.input import observation_input
from stable_baselines.common.policies import nature_cnn

In [5]:
class BnnPolicy(FeedForwardPolicy):
    """
    Policy object that implements actor critic, using a Bayesian neural net (2 layers of 64)
    :param sess: (TensorFlow session) The current TensorFlow session
    :param ob_space: (Gym Space) The observation space of the environment
    :param ac_space: (Gym Space) The action space of the environment
    :param n_env: (int) The number of environments to run
    :param n_steps: (int) The number of steps to run for each environment
    :param n_batch: (int) The number of batch to run (n_envs * n_steps)
    :param reuse: (bool) If the policy is reusable or not
    :param _kwargs: (dict) Extra keyword arguments for the nature CNN feature extraction
    """

    def __init__(self, sess, ob_space, ac_space, n_env, n_steps, n_batch, reuse=False, **_kwargs):
        super(BnnPolicy, self).__init__(sess, ob_space, ac_space, n_env, n_steps, n_batch, reuse,
                                        feature_extraction="bnn", **_kwargs)

# DNN Cartpole

In [6]:
import os
import gym

from stable_baselines.ppo1 import PPO1
from stable_baselines.common.policies import MlpPolicy
from stable_baselines import logger
from stable_baselines.common.callbacks import EvalCallback

NUM_TIMESTEPS = int(1e4)
SEED = 722
EVAL_FREQ = 250000
EVAL_EPISODES = 10  # was 1000

LOGDIR = "dnn_cartpole" # moved to zoo afterwards.
logger.configure(folder=LOGDIR)

env = gym.make("CartPole-v0")
env.seed(SEED)

Logging to dnn_cartpole


[722]

In [7]:
# take mujoco hyperparams (but doubled timesteps_per_actorbatch to cover more steps.)
dnn = PPO1(MlpPolicy, env, timesteps_per_actorbatch=4096, clip_param=0.2, entcoeff=0.0, optim_epochs=10,
                 optim_stepsize=3e-4, optim_batchsize=64, gamma=0.99, lam=0.95, schedule='linear', verbose=2)

eval_callback = EvalCallback(env, best_model_save_path=LOGDIR, log_path=LOGDIR, eval_freq=EVAL_FREQ, n_eval_episodes=EVAL_EPISODES)

dnn.learn(total_timesteps=NUM_TIMESTEPS, callback=eval_callback)

dnn.save(os.path.join(LOGDIR, "final_model")) # probably never get to this point.

env.close()





Instructions for updating:
Use keras.layers.flatten instead.
Instructions for updating:
Please use `layer.__call__` method instead.










Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
********** Iteration 0 ************


/usr/local/anaconda3/envs/slime-rl/lib/python3.7/site-packages/stable_baselines/common/callbacks.py:287: UserWarning: Training and eval env are not of the same type<TimeLimit<CartPoleEnv<CartPole-v0>>> != <stable_baselines.common.vec_env.dummy_vec_env.DummyVecEnv object at 0x7fe007eed850>
  "{} != {}".format(self.training_env, self.eval_env))


Optimizing...
     pol_surr |    pol_entpen |       vf_loss |            kl |           ent
     -0.00983 |       0.00000 |      81.41195 |       0.00399 |       0.68922
     -0.01571 |       0.00000 |      68.77129 |       0.00820 |       0.68506
     -0.01768 |       0.00000 |      43.29578 |       0.00811 |       0.68514
     -0.01988 |       0.00000 |      21.73780 |       0.00893 |       0.68433
     -0.02295 |       0.00000 |      15.19730 |       0.01010 |       0.68317
     -0.02521 |       0.00000 |      13.72938 |       0.01152 |       0.68178
     -0.02703 |       0.00000 |      12.50361 |       0.01233 |       0.68098
     -0.02813 |       0.00000 |      11.25630 |       0.01356 |       0.67978
     -0.02929 |       0.00000 |      10.23180 |       0.01353 |       0.67980
     -0.02959 |       0.00000 |       9.52145 |       0.01405 |       0.67929
Evaluating losses...
     -0.03002 |       0.00000 |       9.25645 |       0.01531 |       0.67806
-----------------------------

# BNN Cartpole

In [8]:
NUM_TIMESTEPS = int(1e4)
SEED = 722
EVAL_FREQ = 250000
EVAL_EPISODES = 10  # was 1000

LOGDIR = "bnn_cartpole" # moved to zoo afterwards.
logger.configure(folder=LOGDIR)

env = gym.make("CartPole-v0")
env.seed(SEED)

Logging to bnn_cartpole


[722]

In [9]:
# take mujoco hyperparams (but doubled timesteps_per_actorbatch to cover more steps.)
bnn = PPO1(BnnPolicy, env, timesteps_per_actorbatch=4096, clip_param=0.2, entcoeff=0.0, optim_epochs=10,
                 optim_stepsize=3e-4, optim_batchsize=64, gamma=0.99, lam=0.95, schedule='linear', verbose=2)

eval_callback = EvalCallback(env, best_model_save_path=LOGDIR, log_path=LOGDIR, eval_freq=EVAL_FREQ, n_eval_episodes=EVAL_EPISODES)

bnn.learn(total_timesteps=NUM_TIMESTEPS, callback=eval_callback)

bnn.save(os.path.join(LOGDIR, "final_model")) # probably never get to this point.

env.close()

Instructions for updating:
Please use `layer.add_weight` method instead.
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
********** Iteration 0 ************


/usr/local/anaconda3/envs/slime-rl/lib/python3.7/site-packages/stable_baselines/common/callbacks.py:287: UserWarning: Training and eval env are not of the same type<TimeLimit<CartPoleEnv<CartPole-v0>>> != <stable_baselines.common.vec_env.dummy_vec_env.DummyVecEnv object at 0x7fdfeb17b090>
  "{} != {}".format(self.training_env, self.eval_env))


Optimizing...
     pol_surr |    pol_entpen |       vf_loss |            kl |           ent
     -0.00126 |       0.00000 |      88.18262 |      7.79e-05 |       0.69307
     -0.00506 |       0.00000 |      85.07277 |       0.00076 |       0.69239
     -0.01154 |       0.00000 |      81.35635 |       0.00365 |       0.68954
     -0.01616 |       0.00000 |      76.96362 |       0.00741 |       0.68586
     -0.01699 |       0.00000 |      71.97631 |       0.00903 |       0.68428
     -0.01771 |       0.00000 |      66.60826 |       0.00836 |       0.68493
     -0.01911 |       0.00000 |      61.00874 |       0.00903 |       0.68426
     -0.02030 |       0.00000 |      55.32611 |       0.00957 |       0.68373
     -0.02151 |       0.00000 |      49.64088 |       0.00998 |       0.68332
     -0.02240 |       0.00000 |      44.18072 |       0.01026 |       0.68305
Evaluating losses...
     -0.02308 |       0.00000 |      41.50755 |       0.00988 |       0.68341
-----------------------------

# Performance

In [10]:
from stable_baselines.common.evaluation import evaluate_policy

mean_reward, std_reward = evaluate_policy(dnn, dnn.get_env(), n_eval_episodes=1000)
print(f"DNN - Mean reward: {mean_reward}, Std reward: {std_reward}")

mean_reward, std_reward = evaluate_policy(bnn, bnn.get_env(), n_eval_episodes=1000)
print(f"BNN - Mean reward: {mean_reward}, Std reward: {std_reward}")

DNN - Mean reward: 187.59, Std reward: 20.69627744305724
BNN - Mean reward: 172.414, Std reward: 27.614210182440488


In [57]:
from stable_baselines.common.evaluation import evaluate_policy

mean_reward, std_reward = evaluate_policy(dnn, dnn.get_env(), n_eval_episodes=1000)
print(f"DNN - Mean reward: {mean_reward}, Std reward: {std_reward}")

mean_reward, std_reward = evaluate_policy(bnn, bnn.get_env(), n_eval_episodes=1000)
print(f"BNN - Mean reward: {mean_reward}, Std reward: {std_reward}")

DNN - Mean reward: 175.151, Std reward: 38.99818199608797
BNN - Mean reward: 164.046, Std reward: 45.66565321989822


In [62]:
from stable_baselines.common.evaluation import evaluate_policy

mean_reward, std_reward = evaluate_policy(dnn, dnn.get_env(), n_eval_episodes=1000)
print(f"DNN - Mean reward: {mean_reward}, Std reward: {std_reward}")

mean_reward, std_reward = evaluate_policy(bnn, bnn.get_env(), n_eval_episodes=1000)
print(f"BNN - Mean reward: {mean_reward}, Std reward: {std_reward}")

DNN - Mean reward: 156.935, Std reward: 48.66798511341928
BNN - Mean reward: 189.304, Std reward: 18.416991719605022


In [67]:
from stable_baselines.common.evaluation import evaluate_policy

mean_reward, std_reward = evaluate_policy(dnn, dnn.get_env(), n_eval_episodes=1000)
print(f"DNN - Mean reward: {mean_reward}, Std reward: {std_reward}")

mean_reward, std_reward = evaluate_policy(bnn, bnn.get_env(), n_eval_episodes=1000)
print(f"BNN - Mean reward: {mean_reward}, Std reward: {std_reward}")

DNN - Mean reward: 161.503, Std reward: 47.323144348193935
BNN - Mean reward: 178.989, Std reward: 32.992982268961384


# Evalutaion

There are a few possible ways we can take this:
1. Small number of iterations on CartPole, to see which model performs better with less data.
2. Run experiment on Acrobot, to hopefully see BNN > DNN.
3. Hide 1 out of every N frames from the learning agent (self-play or single-agent), to hopefully see BNN > DNN.

For all of the above, run a hundred different initialisations using different random seeds, and evaluate using at least 1K episodes (the episodes have extremely high variance even with the same random seed)

In [18]:
import pandas as pd

#To store experiment results
df = pd.DataFrame(columns = ['Seed', 'BNN reward', 'BNN std', 'DNN reward', 'DNN std']) 

df.head()

,Seed,BNN reward,BNN std,DNN reward,DNN std


In [21]:
#Evaluation method one

import os
import gym

from stable_baselines.ppo1 import PPO1
from stable_baselines.common.policies import MlpPolicy
from stable_baselines import logger
from stable_baselines.common.callbacks import EvalCallback
from stable_baselines.common.evaluation import evaluate_policy

NUM_TIMESTEPS = int(1e4)
SEED = 722
EVAL_FREQ = 250000
EVAL_EPISODES = 10  # was 1000

for SEED in range (600, 701):
    LOGDIR = "dnn_cartpole" # moved to zoo afterwards.
    logger.configure(folder=LOGDIR)
    
    env = gym.make("CartPole-v0")
    env.seed(SEED)

    # take mujoco hyperparams (but doubled timesteps_per_actorbatch to cover more steps.)
    dnn = PPO1(MlpPolicy, env, timesteps_per_actorbatch=4096, clip_param=0.2, entcoeff=0.0, optim_epochs=10,
                 optim_stepsize=3e-4, optim_batchsize=64, gamma=0.99, lam=0.95, schedule='linear', verbose=2)

    eval_callback = EvalCallback(env, best_model_save_path=LOGDIR, log_path=LOGDIR, eval_freq=EVAL_FREQ, n_eval_episodes=EVAL_EPISODES)

    dnn.learn(total_timesteps=NUM_TIMESTEPS, callback=eval_callback)

    dnn.save(os.path.join(LOGDIR, "final_model")) # probably never get to this point.

    env.close()
    
    LOGDIR = "bnn_cartpole" # moved to zoo afterwards.
    logger.configure(folder=LOGDIR)

    env = gym.make("CartPole-v0")
    env.seed(SEED)
    
    # take mujoco hyperparams (but doubled timesteps_per_actorbatch to cover more steps.)
    bnn = PPO1(BnnPolicy, env, timesteps_per_actorbatch=4096, clip_param=0.2, entcoeff=0.0, optim_epochs=10,
                 optim_stepsize=3e-4, optim_batchsize=64, gamma=0.99, lam=0.95, schedule='linear', verbose=2)

    eval_callback = EvalCallback(env, best_model_save_path=LOGDIR, log_path=LOGDIR, eval_freq=EVAL_FREQ, n_eval_episodes=EVAL_EPISODES)

    bnn.learn(total_timesteps=NUM_TIMESTEPS, callback=eval_callback)

    bnn.save(os.path.join(LOGDIR, "final_model")) # probably never get to this point.

    env.close()
    
    mean_rewardDNN, std_rewardDNN = evaluate_policy(dnn, dnn.get_env(), n_eval_episodes=1000)
    print(f"DNN - Mean reward: {mean_rewardDNN}, Std reward: {std_rewardDNN}")

    mean_rewardBNN, std_rewardBNN = evaluate_policy(bnn, bnn.get_env(), n_eval_episodes=1000)
    print(f"BNN - Mean reward: {mean_rewardBNN}, Std reward: {std_rewardBNN}")
    
    data = {
    "Seed" : SEED,
    "BNN reward" : mean_rewardBNN,
    "BNN std" : std_rewardBNN,
    "DNN reward" : mean_rewardDNN,
    "DNN std" : std_rewardDNN
    }

    df = df.append(data, ignore_index=True)

    

Logging to dnn_cartpole
********** Iteration 0 ************


/usr/local/anaconda3/envs/slime-rl/lib/python3.7/site-packages/stable_baselines/common/callbacks.py:287: UserWarning: Training and eval env are not of the same type<TimeLimit<CartPoleEnv<CartPole-v0>>> != <stable_baselines.common.vec_env.dummy_vec_env.DummyVecEnv object at 0x7fdfece47f90>
  "{} != {}".format(self.training_env, self.eval_env))


Optimizing...
     pol_surr |    pol_entpen |       vf_loss |            kl |           ent
     -0.01104 |       0.00000 |      83.31959 |       0.00455 |       0.68846
     -0.01737 |       0.00000 |      70.39660 |       0.00789 |       0.68506
     -0.01893 |       0.00000 |      44.48908 |       0.00881 |       0.68414
     -0.02156 |       0.00000 |      22.42776 |       0.00901 |       0.68393
     -0.02429 |       0.00000 |      15.77996 |       0.01042 |       0.68252
     -0.02641 |       0.00000 |      14.44719 |       0.01168 |       0.68127
     -0.02701 |       0.00000 |      13.38486 |       0.01305 |       0.67991
     -0.02805 |       0.00000 |      12.24819 |       0.01337 |       0.67960
     -0.02847 |       0.00000 |      11.23798 |       0.01376 |       0.67921
     -0.02903 |       0.00000 |      10.48966 |       0.01386 |       0.67911
Evaluating losses...
     -0.02881 |       0.00000 |      10.17848 |       0.01603 |       0.67697
-----------------------------

/usr/local/anaconda3/envs/slime-rl/lib/python3.7/site-packages/stable_baselines/common/callbacks.py:287: UserWarning: Training and eval env are not of the same type<TimeLimit<CartPoleEnv<CartPole-v0>>> != <stable_baselines.common.vec_env.dummy_vec_env.DummyVecEnv object at 0x7fdfee5b4090>
  "{} != {}".format(self.training_env, self.eval_env))


Optimizing...
     pol_surr |    pol_entpen |       vf_loss |            kl |           ent
     -0.00168 |       0.00000 |      93.04918 |       0.00010 |       0.69305
     -0.00627 |       0.00000 |      90.33788 |       0.00093 |       0.69221
     -0.01293 |       0.00000 |      87.07071 |       0.00436 |       0.68881
     -0.01600 |       0.00000 |      83.06079 |       0.00770 |       0.68553
     -0.01655 |       0.00000 |      78.25071 |       0.00884 |       0.68440
     -0.01734 |       0.00000 |      72.83126 |       0.00866 |       0.68458
     -0.01846 |       0.00000 |      67.07480 |       0.00882 |       0.68442
     -0.01998 |       0.00000 |      61.22139 |       0.00917 |       0.68406
     -0.02124 |       0.00000 |      55.44053 |       0.00942 |       0.68382
     -0.02289 |       0.00000 |      49.86128 |       0.01027 |       0.68297
Evaluating losses...
     -0.02381 |       0.00000 |      47.13252 |       0.01006 |       0.68318
-----------------------------

...

Optimizing...
     pol_surr |    pol_entpen |       vf_loss |            kl |           ent
     -0.00160 |       0.00000 |      83.75403 |       0.00017 |       0.69298
     -0.00666 |       0.00000 |      80.94183 |       0.00212 |       0.69104
     -0.01119 |       0.00000 |      77.86915 |       0.00500 |       0.68821
     -0.01466 |       0.00000 |      74.15305 |       0.00636 |       0.68686
     -0.01756 |       0.00000 |      69.73478 |       0.00834 |       0.68491
     -0.01864 |       0.00000 |      64.78265 |       0.00940 |       0.68387
     -0.01989 |       0.00000 |      59.52032 |       0.00931 |       0.68395
     -0.02097 |       0.00000 |      54.04965 |       0.00948 |       0.68379
     -0.02208 |       0.00000 |      48.68857 |       0.01029 |       0.68299
     -0.02362 |       0.00000 |      43.51255 |       0.01068 |       0.68260
Evaluating losses...
     -0.02396 |       0.00000 |      41.03507 |       0.01060 |       0.68267
-----------------------------

/usr/local/anaconda3/envs/slime-rl/lib/python3.7/site-packages/stable_baselines/common/callbacks.py:287: UserWarning: Training and eval env are not of the same type<TimeLimit<CartPoleEnv<CartPole-v0>>> != <stable_baselines.common.vec_env.dummy_vec_env.DummyVecEnv object at 0x7fdf4885f790>
  "{} != {}".format(self.training_env, self.eval_env))


Optimizing...
     pol_surr |    pol_entpen |       vf_loss |            kl |           ent
     -0.00910 |       0.00000 |      93.43211 |       0.00403 |       0.68903
     -0.01557 |       0.00000 |      79.82188 |       0.00767 |       0.68535
     -0.01689 |       0.00000 |      51.85278 |       0.00791 |       0.68511
     -0.01915 |       0.00000 |      26.54080 |       0.00852 |       0.68450
     -0.02218 |       0.00000 |      18.01200 |       0.00939 |       0.68363
     -0.02459 |       0.00000 |      16.29358 |       0.01075 |       0.68228
     -0.02639 |       0.00000 |      15.16594 |       0.01215 |       0.68090
     -0.02806 |       0.00000 |      13.92322 |       0.01273 |       0.68032
     -0.02870 |       0.00000 |      12.81259 |       0.01323 |       0.67983
     -0.02950 |       0.00000 |      12.00123 |       0.01352 |       0.67955
Evaluating losses...
     -0.02929 |       0.00000 |      11.66571 |       0.01159 |       0.68145
-----------------------------

/usr/local/anaconda3/envs/slime-rl/lib/python3.7/site-packages/stable_baselines/common/callbacks.py:287: UserWarning: Training and eval env are not of the same type<TimeLimit<CartPoleEnv<CartPole-v0>>> != <stable_baselines.common.vec_env.dummy_vec_env.DummyVecEnv object at 0x7fdf8730e6d0>
  "{} != {}".format(self.training_env, self.eval_env))


Optimizing...
     pol_surr |    pol_entpen |       vf_loss |            kl |           ent
     -0.00178 |       0.00000 |      92.79841 |       0.00018 |       0.69297
     -0.00709 |       0.00000 |      90.34549 |       0.00149 |       0.69167
     -0.01435 |       0.00000 |      87.34417 |       0.00540 |       0.68781
     -0.01657 |       0.00000 |      83.58577 |       0.00800 |       0.68526
     -0.01696 |       0.00000 |      79.13976 |       0.00896 |       0.68432
     -0.01777 |       0.00000 |      74.19498 |       0.00907 |       0.68422
     -0.01868 |       0.00000 |      68.83524 |       0.00895 |       0.68433
     -0.02013 |       0.00000 |      63.23062 |       0.00956 |       0.68373
     -0.02147 |       0.00000 |      57.58737 |       0.00981 |       0.68348
     -0.02266 |       0.00000 |      51.98813 |       0.01021 |       0.68307
Evaluating losses...
     -0.02332 |       0.00000 |      49.20932 |       0.01067 |       0.68263
-----------------------------

In [22]:
df.head()

,Seed,BNN reward,BNN std,DNN reward,DNN std
0,600.0,189.033,19.244737,181.186,30.742241
1,600.0,174.573,38.308180,173.827,39.677715
2,601.0,158.712,54.098254,174.281,37.204463
3,602.0,173.205,37.427329,183.545,26.022298
4,603.0,178.991,33.860403,167.562,44.853006


In [23]:
#export dataframe as csv so you dont have to run experiment again
df.to_csv(r'BNNCartpoleExperiment1.csv', index = False)

# Experiment Analysys

In [ ]:
#load experiment data from csv
